In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords 

from numpy import array 
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM
from keras.layers import Input,RepeatVector,Permute,Multiply,Lambda,Bidirectional
from keras import backend as K
from keras.models import Model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
df = pd.read_csv('reviews.csv')
df.isnull().values.any() # checks if data file have any null values
df.head()

,reviews,sentiment
0,"Well, you'd better if you plan on sitting thro...",0
1,Moonwalker is a Fantasy Music film staring Mic...,1
2,I bought this video on a throw-out table at th...,0
3,Since the last horrid Astérix film and the fac...,1
4,"I grew up with the Abbott and Costello movies,...",1


In [3]:
def preprocess(text):
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
  text = re.sub('[\W]+', ' ', text.lower()) +\
      ' '.join(emoticons).replace('-', '')
  return text
df.shape

(50000, 2)

In [4]:
X = []
sentences = list(df['reviews'])
for sen in sentences:
    X.append(preprocess(sen))
y = df['sentiment']

In [5]:
X[0]

'well you d better if you plan on sitting through this amateurish bland and pokey flick about a middle aged widowed mom who has a little more in common with her young adult or old teen daughter than she would like set in tunis mom piddles around the flat gets antsy and decides albeit reluctantly she just can t help herself to don the costume and dance in a local cabaret meanwhile her daughter is taking dancing lessons the common denominator is a tunisian band drummer this film is so full of filler i watched the dvd at x2 and read the subtitles fast forwarding through much of the very ordinary dancing and loooong shots of walking they walk everywhere and more walking and just plain dawdling at x4 just to get though this boring uneventful low budget flick which some how garnered some pretty good critical plaudits go figure c '

# Train( 80 ) Test( 20 ) Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Embedding Layer ( For converting textual data into numeric data )

* First layer in DL model in Keras

In [7]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(len(X_train[0]))
print(len(X_test[0]))

180
244


In [8]:
# padding process 
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [9]:
print(len(X_train[0]))
print(len(X_test[0]))

100
100


In [10]:
print(vocab_size) # number of unique words in dataset

94562


In [11]:
# using GloVe for creating feature matrix
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.300d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [31]:
embedding_matrix = zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [36]:
embedding_matrix.shape
vocab_size = embedding_matrix.shape[0]
print(vocab_size)

94562


# Text Classification with 2 LSTM Layer

In [33]:
# '''model = Sequential()
# model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False))
# model.add( LSTM( 512,return_sequences=True))
# model.add( LSTM( 512, return_sequences=False))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])'''

"model = Sequential()\nmodel.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False))\nmodel.add( LSTM( 512,return_sequences=True))\nmodel.add( LSTM( 512, return_sequences=False))\nmodel.add(Dense(1,activation='sigmoid'))\nmodel.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])"

In [ ]:
from keras.layers import LSTM
from keras.layers import Bidirectional
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen))
model.add(Bidirectional(LSTM(200)))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [37]:
# units=200
# vocab_size = embedding_matrix.shape[0]
# embedding_size = embedding_matrix.shape[1]


# _input = Input(shape=[maxlen], dtype='int32')

# # get the embedding layer
# embedded = Embedding(
#         input_dim=vocab_size,
#         output_dim=embedding_size,
#         input_length=maxlen,
#         trainable=False,
#         mask_zero=False,
#         weights=[embedding_matrix]
#     )(_input)

# activations = LSTM(units, return_sequences=True)(embedded)
# activations = Dropout(0.4)(activations)


# # compute importance for each step
# attention = Dense(1, activation='tanh')(activations) 
# attention = Flatten()(attention)
# attention = Activation('softmax')(attention)
# attention = RepeatVector(units)(attention)
# attention = Permute([2, 1])(attention)

# # apply the attention
# sent_representation = Multiply()([activations, attention])
# sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(units,))(sent_representation)
# sent_representation = Dropout(0.4)(sent_representation)

# probabilities = Dense(1, activation='sigmoid')(sent_representation)

# model = Model(_input,probabilities)
# checkpoint = ModelCheckpoint(filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5", verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
# callbacks_list = [checkpoint]
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [39]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data = (X_test,y_test))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 110s 3ms/step - loss: 0.2898 - accuracy: 0.8755 - val_loss: 0.3284 - val_accuracy: 0.8526
Epoch 2/10
40000/40000 [==============================] - 113s 3ms/step - loss: 0.2693 - accuracy: 0.8861 - val_loss: 0.3310 - val_accuracy: 0.8532
Epoch 3/10
40000/40000 [==============================] - 112s 3ms/step - loss: 0.2471 - accuracy: 0.8941 - val_loss: 0.3294 - val_accuracy: 0.8579
Epoch 4/10
40000/40000 [==============================] - 115s 3ms/step - loss: 0.2207 - accuracy: 0.9093 - val_loss: 0.3366 - val_accuracy: 0.8547
Epoch 5/10
12608/40000 [========>.....................] - ETA: 1:11 - loss: 0.1911 - accuracy: 0.9227

KeyboardInterrupt: 

In [19]:
model.save("Senti97.h5py")

In [1]:
from keras.models import load_model
model=load_model("Senti97.h5py")

Using TensorFlow backend.


ValueError: Unknown layer:name

In [17]:
x='is the rare movie that succeeds on almost every level, where each character, scene, costume, and joke firing on all cylinders to make a film worth repeated viewings.'

In [18]:
X = []
X.append(preprocess(x))

In [19]:
X = tokenizer.texts_to_sequences(X)

In [20]:
X=pad_sequences(X, padding='post', maxlen=maxlen)

In [21]:
model.predict_proba(X)

array([[0.5952437]], dtype=float32)

In [22]:
model.predict_classes(X)

array([[1]])